# LCEL 中添加 memory

In [1]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_ollama.chat_models import ChatOllama

model = ChatOllama(model="qwen2.5:7b")
prompt = ChatPromptTemplate.from_messages(
  [
    ("system", "你是一个乐于助人的机器人"),
    MessagesPlaceholder(variable_name="history"),
    ("human","{input}"),
  ]
)

In [3]:
memory = ConversationBufferMemory(return_messages=True)
memory.load_memory_variables(inputs={})

{'history': []}

In [6]:
# 增加一条链
# 创建一个链式调用的链对象
# 1. 使用 RunnablePassthrough.assign 方法为链分配变量
#    - history: 通过 RunnableLambda 调用 memory.load_memory_variables 方法加载内存变量
#    - 然后通过 itemgetter('history') 提取 'history' 键对应的值
# 2. 将分配的变量传递给 prompt 模板
# 3. 最后将 prompt 的输出传递给 model 进行处理

chain = (
  RunnablePassthrough.assign(
    history=RunnableLambda(memory.load_memory_variables) | itemgetter('history')
  )
  | prompt | model
)

In [11]:
inputs = {"input":"你好我是GGBom"}

In [12]:
response =  chain.invoke(inputs)

In [13]:
# 保存记忆
memory.save_context(inputs, {"outputs":response.content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='你好我是GGBom', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好！GGBom，很高兴认识你。你可以告诉我更多关于你自己或者你需要帮助的一些事情吗？我会尽力提供帮助。', additional_kwargs={}, response_metadata={})]}

In [14]:
inputs = {"input":"我是谁？"}
response = chain.invoke(inputs)
response.content

'你在自我介绍时说你是“GGBom”，这个名字可能是指你的昵称或者是某个特定角色/账号的名字。在我们之前的对话中，你是想要了解自己是谁吗？还是有其他问题呢？如果你能提供更多背景信息，我会更好地帮助你。'

## 使用Redis来实现长期记忆

In [ ]:
! pip install redis

### Redis
Redis 是一个开源的内存数据结构存储，用作数据库、缓存和消息代理。它支持多种数据结构，如字符串、哈希、列表、集合和有序集合。Redis 通常用于需要快速访问和存储数据的场景。

在本项目中，我们使用 Redis 来实现长期记忆功能。通过将聊天记录存储在 Redis 中，可以在不同会话之间共享和持久化聊天历史。

以下是 Redis 的基本使用步骤：
1. 安装 Redis 并启动服务。
2. 使用 Redis 客户端连接到 Redis 服务器。
3. 执行基本的 Redis 命令，如 `SET` 和 `GET`，来存储和检索数据。
4. 在本项目中，通过 `RedisChatMessageHistory` 类与 Redis 集成，实现聊天记录的存储和管理。

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableWithMessageHistory

model = ChatOllama(model="qwen2.5:7b")
prompt = ChatPromptTemplate.from_messages(
  [
    ("system", "你是一个善于{dowhat}的机器人"),
    MessagesPlaceholder(variable_name="history"),
    ("human","{input}"),
  ]
)

chain_with_histroy = RunnableWithMessageHistory(
  chain,
  # 使用 Redis 存储聊天记录
  lambda session_id: RedisChatMessageHistory(session_id, url="redis://localhost:6379/0"),
  input_messages_key="question",
  tory_messages_key="history"
)